In [1]:
#Saturated Oxygen Data Correction
import oxyfloat

In [2]:
oga_floats = oxyfloat.get_oxy_floats()

DEBUG 2015-07-29 08:22:06,053 __init__.py get_oxy_floats():34 Reading data from http://argo.jcommops.org/FTPRoot/Argo/Status/argo_all.txt
DEBUG:oxyfloat:Reading data from http://argo.jcommops.org/FTPRoot/Argo/Status/argo_all.txt
DEBUG 2015-07-29 08:22:25,990 __init__.py get_oxy_floats():52 Merging records with oxygen, not greylisted, and age >= 340
DEBUG:oxyfloat:Merging records with oxygen, not greylisted, and age >= 340


In [ ]:
for dac_url in oxyfloat.get_dac_urls(oga_floats):
    for profile_url in oxyfloat.get_profile_opendap_urls(dac_url):
        print 'Reading data from ...%s', profile_url[40:]
        try:
            float_data = oxyfloat.get_profile_data(profile_url)
        except oxyfloat.RequiredVariableNotPresent as e:
            logger.warn(e)
        else:
            print float_data


DEBUG 2015-07-29 08:26:23,895 __init__.py get_dac_urls():66 Reading data from ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt
DEBUG:oxyfloat:Reading data from ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt
DEBUG 2015-07-29 08:26:28,493 __init__.py get_dac_urls():83 Found 555 dac_urls
DEBUG:oxyfloat:Found 555 dac_urls
DEBUG 2015-07-29 08:26:28,498 __init__.py get_profile_opendap_urls():94 Parsing http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/1900722/profiles/catalog.xml
DEBUG:oxyfloat:Parsing http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/1900722/profiles/catalog.xml
DEBUG 2015-07-29 08:26:29,602 __init__.py get_profile_opendap_urls():116 Found 405 netCDF files in 1.1 seconds
DEBUG:oxyfloat:Found 405 netCDF files in 1.1 seconds
DEBUG 2015-07-29 08:26:29,607 __init__.py get_profile_data():123 Opening http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/aoml/1900722/profiles/MD1900722_135.nc
DEBUG:oxyfloat:Opening http://thredds.aodn

The above cell failed with a chashing of the kernel at the 'ds = Dataset(url)' call.

The code below is form the old notebook. Some of it may be folded into oxyfloat functions.

In [ ]:
floatLookup = {}
for url in urls:
    print url
    ds = Dataset(url)
    floatNum = url.split('/')[9]
    print floatNum
    p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], 
               ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])
    lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])
    
    # Convert o from micromol/kg to ml/l 
    ##perc_sat = 100 * convertTomll(o) / satO2(s,t) 
    print s,t,perc_sat
    floatLookup[floatNum] = (p,t,s,o,lat,lon,mtime)

In [ ]:
floatLookup

In [33]:
p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])

In [53]:
p,t,s,o

(4.3000002, 29.790001, 34.827457, 206.49001)

In [35]:
lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])

In [36]:
lat,lon,mtime

(6.0310000000000006, -14.026, 21985.069212962964)

In [118]:
#The p here is for depth...maybe
#That's why it said the unit was km
def dens_formula(p , t , s):
    C = (999.83 + (5.053*p) - (.048*(p**2))
    B =  .808 - (.0085*p)
    a = .0708*((1 + (.351*p)) + .068*(1 - (.0683*p))*t)
    y = .003*(1 - (.059*p) - .012*(1 - .064*p)*t)
    dens = C + (B*s) - (a*t) - ((y*(35 - s))*t)
    return dens

SyntaxError: invalid syntax (<ipython-input-118-819d3bb21dd0>, line 5)

In [115]:
dens_formula(p , t , s)

1039.220347679988

In [ ]:
def dens_formula(p , t , s):
    p = p / 10.0
    pw = 999.842594 + ((6.793952 * (10**-abs(2)))*t) - ((9.095290 * (10**-abs(3)))*(t**2)) + ((1.001685 * (10**-abs(4)))*(t**3)) - ((1.120083 * (10**-abs(6)))*(t**4)) + ((6.536332 * (10**-abs(9))*(t**5))        
    